In [30]:
#import only labels from /Users/nicholasconterno/Fishnet3/data/fishnet_labels.csv 
import pandas as pd
#import file
labels = pd.read_csv('../data/fishnet_labels.csv')
labels.head()

,img_id,cam_id,seq_id,frame_id,bbox_id,x_min,x_max,y_min,y_max,label_l1,label_l2,fao_mfa,train,val,test
0,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,1,626,754,-1,88,Human,HUMAN,81,True,False,False
1,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,3,544,710,15,203,Human,HUMAN,81,True,False,False
2,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,4,629,735,210,562,Human,HUMAN,81,True,False,False
3,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,5,615,883,550,907,Human,HUMAN,81,True,False,False
4,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,6,885,962,826,875,Unknown,OTH,81,True,False,False


In [31]:
#read from uploadedfiles.txt
# Path to your text file
file_path = '../uploadedfiles.txt'

# Initialize an empty list to store the URLs
url_list = []

# Open the file and read line by line
with open(file_path, 'r',encoding='utf-16') as file:
    for line in file:
        # Strip newline and whitespace characters and then add to the list
        url = line.strip()
        if url:  # Check if the line is not empty
            fileName = url.split('fishnet3-object-detection/')[1][:-4]
            url_list.append(fileName)
img_id_list= url_list[1:]


In [32]:


# Filter the DataFrame to keep rows where img_id is in img_id_list
filtered_df = labels[labels['img_id'].isin(img_id_list)]
fishes_to_remove = ['Mola mola', 'Roudie scolar', 'Snake mackerel', 'Rainbow runner', 'Thresher shark', 'Brama']
filtered_df = filtered_df[~filtered_df['label_l1'].isin(fishes_to_remove)]

#add a column is_fish to the filtered_df
no_fish = ['Human', 'Unknown', 'No fish', 'Water']
filtered_df['is_fish'] = ~filtered_df['label_l1'].isin(no_fish)
# Path for the new CSV file
new_csv_file_path = '../data/fishnet_labels_in_bucket.csv'

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv(new_csv_file_path, index=False)

print("Filtered CSV saved as:", new_csv_file_path)


Filtered CSV saved as: ../data/fishnet_labels_in_bucket.csv


In [33]:
#read pandas from csv
df = pd.read_csv('../data/fishnet_labels_in_bucket.csv')
# print number of unique img_id
print("Number of unique img_id:", df['img_id'].nunique())




Number of unique img_id: 115614


In [34]:
import sys

# Replace 'path/to/my_folder' with the path to the folder containing your script
folder_path = '../data-preprocessing/'
sys.path.append(folder_path)

# Now import the specific function from the script
from label_parser import process_data

imageSet = process_data('../data/fishnet_labels_in_bucket.csv')
count = 0
for key in imageSet.keys():
    if count < 10:
        print(key, (imageSet[key]))
        count += 1



da7c5ef0-db28-11ea-b693-9b3523cc10a5 [False, [(('626', '754'), ('-1', '88'), 'Human'), (('544', '710'), ('15', '203'), 'Human'), (('629', '735'), ('210', '562'), 'Human'), (('615', '883'), ('550', '907'), 'Human'), (('885', '962'), ('826', '875'), 'Unknown')]]
da7b73f0-db28-11ea-b585-db222369d02d [False, [(('264', '440'), ('319', '625'), 'Human'), (('600', '728'), ('-2', '177'), 'Human'), (('583', '781'), ('250', '590'), 'Human'), (('493', '626'), ('457', '732'), 'Human'), (('603', '851'), ('560', '980'), 'Human'), (('914', '1002'), ('908', '1029'), 'Unknown')]]
da4b3cee-db28-11ea-b10d-bbe56eb82b48 [True, [(('38', '194'), ('0', '198'), 'Human'), (('1178', '1342'), ('86', '278'), 'Human'), (('1153', '1254'), ('253', '344'), 'Yellowfin tuna')]]
94dd66c8-23f0-11e9-b4f7-7feec33e644c [True, [(('0', '183'), ('392', '669'), 'Human'), (('168', '343'), ('81', '407'), 'Human'), (('78', '376'), ('542', '666'), 'Albacore'), (('0', '126'), ('3', '141'), 'Human')]]
fc4bdea8-5952-11ec-97e5-dbbed9401b

In [35]:
import random

# Assuming 'data_map' is your dictionary
true_ids = [img_id for img_id, data in imageSet.items() if data[0]]
false_ids = [img_id for img_id, data in imageSet.items() if not data[0]]
for id in true_ids[:5]:
    print(id, imageSet[id])
for id in false_ids[:5]:   
    print(id, imageSet[id])
# Randomly select 5000 IDs from each list
selected_true_ids = random.sample(true_ids, 5000)
selected_false_ids = random.sample(false_ids, 5000)

# Combine the lists
selected_ids = selected_true_ids + selected_false_ids

# Shuffle the combined list to mix True and False IDs
random.shuffle(selected_ids)

# Save the IDs to a file in json format
import json
with open('../data/imagefilenames.json', 'w') as file:
    json.dump(selected_ids, file)
#update fishnet_labels_in_bucket.csv to only include the selected_ids
filtered_df = df[df['img_id'].isin(selected_ids)]
# Path for the new CSV file
new_csv_file_path = '../data/fishnet_labels_FINAL.csv'
# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv(new_csv_file_path, index=False)
print("Filtered CSV saved as:", new_csv_file_path)



print("Saved 10,000 image IDs to imageIDs.txt")


da4b3cee-db28-11ea-b10d-bbe56eb82b48 [True, [(('38', '194'), ('0', '198'), 'Human'), (('1178', '1342'), ('86', '278'), 'Human'), (('1153', '1254'), ('253', '344'), 'Yellowfin tuna')]]
94dd66c8-23f0-11e9-b4f7-7feec33e644c [True, [(('0', '183'), ('392', '669'), 'Human'), (('168', '343'), ('81', '407'), 'Human'), (('78', '376'), ('542', '666'), 'Albacore'), (('0', '126'), ('3', '141'), 'Human')]]
fc4bdea8-5952-11ec-97e5-dbbed9401b90 [True, [(('71', '222'), ('209', '354'), 'Yellowfin tuna')]]
94b080fe-23f0-11e9-ae46-fbc21443401c [True, [(('679', '1029'), ('0', '214'), 'Human'), (('1016', '1279'), ('323', '518'), 'Human'), (('46', '546'), ('14', '344'), 'Human'), (('315', '686'), ('372', '543'), 'Albacore'), (('844', '1134'), ('484', '591'), 'Albacore')]]
fc4b5cb2-5952-11ec-96c3-4b67933b0949 [True, [(('178', '455'), ('584', '726'), 'Yellowfin tuna'), (('304', '558'), ('401', '543'), 'Yellowfin tuna'), (('178', '455'), ('584', '726'), 'Yellowfin tuna'), (('304', '558'), ('401', '543'), 'Yell

In [36]:
imageSet = process_data('../data/fishnet_labels_FINAL.csv')
print("Number of unique img_id:", len(imageSet))


Number of unique img_id: 10000


In [39]:
#print 5 key value pairs
count = 0
for key in imageSet.keys():
    if count < 5:
        print(key, (imageSet[key]))
        count += 1
print("Number of unique img_id:", len(imageSet))

da9120a6-db28-11ea-9d63-ef6f2e9f3726 [False, [(('196', '314'), ('688', '749'), 'Human'), (('561', '801'), ('484', '749'), 'Human'), (('918', '1108'), ('586', '749'), 'Human')]]
da4e942a-db28-11ea-b98f-df975a039489 [True, [(('0', '234'), ('0', '404'), 'Human'), (('140', '460'), ('0', '340'), 'Human'), (('846', '1121'), ('111', '245'), 'Swordfish'), (('1097', '1236'), ('10', '204'), 'Human')]]
dac59b6a-db28-11ea-bce5-ef781d7c8eab [True, [(('620', '868'), ('192', '333'), 'Albacore'), (('690', '889'), ('0', '245'), 'Human')]]
94da7cec-23f0-11e9-b15f-37a875ab36ff [False, [(('316', '657'), ('0', '520'), 'Human')]]
fc71ac50-5952-11ec-b6a8-cff8ba66b119 [True, [(('360', '432'), ('281', '365'), 'Albacore')]]
Number of unique img_id: 10000


In [40]:
# Convert the dictionary to a JSON string
json_data = json.dumps(imageSet, indent=4)

# Save the JSON data to a file
with open('../data/labels.json', 'w') as file:
    file.write(json_data)